In [14]:
import pandas as pd

# init empty lists
data = []
EPC_sep = []
RSSI_min = []
data_new = []
EPC_count_list = []

#init empty dictionary
mapping = {}

# load csv file into dataframe and change header row (deleted everything before)
input = 'Data/REV7/6 Rick Brophy/RB 3m/1 Lat Raise/2024-07-29_10-38-10.csv'
    #print(f'Trying to read csv file: {input}')
data.append(pd.read_csv(input, header = 2))
    
for i in range(len(data)):
    # change which columns are needed and remove spaces
    data[i] = data[i].iloc[0:, [0,1,4,7]]
    data[i].columns = data[i].columns.str.strip()

    # rename columns
    data[i].rename(columns={'// Timestamp' : 'Timestamp'},     inplace = True)
    #data[i].rename(columns={' RSSI'        : 'RSSI'},          inplace = True)
    #data[i].rename(columns={' PhaseAngle'  : 'PhaseAngle'},    inplace = True)
    #data[i].rename(columns={' EPC'         : 'EPC'},           inplace = True)

    '''
    # parse timestamp data and create new column to have comparable time values
    data[i]['Timestamp'] = pd.to_datetime(data[i]['Timestamp'])
    first_timestamp = data[i]['Timestamp'].iloc[0]
    data[i]['TimeElapsed'] = data[i]['Timestamp'] - first_timestamp

    # separate time elapsed into seconds and milliseconds, then combine the numbers
    data[i]['seconds'] = data[i]['TimeElapsed'].dt.seconds
    data[i]['milliseconds'] = data[i]['TimeElapsed'].dt.microseconds // 1000
    data[i]['TimeValue'] = data[i]['seconds'] + data[i]['milliseconds'] / 1000
    data[i]['TimeValue'] = data[i]['TimeValue'].round(4) # round to 4 decimal places

    # move timevalue column over
    data[i]['Timestamp'] = data[i]['TimeValue']
    data[i].rename(columns={'Timestamp': 'TimeValue'}, inplace=True)
    '''

    # parse timestamp data and create new column to have comparable time values
    data[i]['Timestamp'] = pd.to_datetime(data[i]['Timestamp'])
    first_timestamp = data[i]['Timestamp'].iloc[0]
    data[i]['Timestamp'] = (data[i]['Timestamp'] - first_timestamp).dt.total_seconds()

    # move timevalue column over
    #data[i]['Timestamp'] = data[i]
    #data[i].drop(columns=['Timestamp'], inplace=True)
    data[i].rename(columns={'Timestamp': 'TimeValue'}, inplace=True)
    print(data[i].columns)
    print(data[i])

Index(['TimeValue', 'EPC', 'RSSI', 'PhaseAngle'], dtype='object')
     TimeValue                       EPC   RSSI         PhaseAngle
0     0.000000  A40000000000000000000000  -66,5  4,933282213840222
1     0.001383  A80000000000000000000000  -66,5  4,816699673960914
2     0.005240  A90000000000000000000000    -67  5,062136600022616
3     0.008158  A50000000000000000000000  -59,5  0,944932165337555
4     0.009645  A30000000000000000000000  -65,5   4,73693267299086
..         ...                       ...    ...                ...
143   2.628222  A90000000000000000000000    -67  5,258486140871978
144   2.629481  A80000000000000000000000  -68,5  5,240078371417351
145   2.631117  A40000000000000000000000  -65,5  2,325514874434632
146   2.634174  A60000000000000000000000    -69  4,147884050442774
147   2.640513  A20000000000000000000000    -72  4,853515212870168

[148 rows x 4 columns]
